In [ ]:
# imports
import pathlib
import datetime
import random
import numpy as np
import json
import torch
import torch.nn as nn

from tqdm import tqdm
from IPython.display import clear_output
from model import NCA_model
from utils import Utils

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def show_image(img):
    #print ('img.shape: ', img.shape)
    img_rgb = Utils.to_rgb(img).squeeze().permute(1, 2, 0)
    plt.axis('off')
    plt.imshow(img_rgb)
    plt.show()

def show_batch(batch):
    batch_size = batch.shape[0]
    f, axarr = plt.subplots(1, batch_size, figsize=(32, 32))
    
    for i in range(batch_size):
        img_i = batch[i]
        img_i = img_i[:4].unsqueeze(0)
        img_rgb = Utils.to_rgb(img_i).squeeze().permute(1, 2, 0)
        axarr[i].axis('off')
        axarr[i].imshow(img_rgb)
        axarr[i].set_title(str(i))
    plt.show()

In [ ]:
# code block used for experiments

import sys
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(profile="full")

x = torch.rand(2, 3, 3, 3)
#print (x)

z = torch.zeros(1, 3, 3, 1)

x[-1:] *= z
#print (x)

In [ ]:
# training parameters
img = 'imgs\pup.png'
name = 'pup40_dam3'
save_model = True
viz_train = True

size = 40
pad = 16
n_channels = 16

n_train_iter = 8000
batch_size = 8
pool_size = 1024
n_damage = 3

eval_freq = 500
eval_iter = 300

log_dir = 'logs'
model_dir = 'models'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print ('cuda available? ', torch.cuda.is_available())
print ('device: ', device)

In [ ]:
# misc
full_size = size + (2 * pad)
#device_n = torch.device(device)

# create log
log_path = pathlib.Path(log_dir)
log_path.mkdir(parents=True, exist_ok=True)

# target image
loaded_img = Utils.load_image(img, size)
target_img_ = nn.functional.pad(loaded_img, (pad, pad, pad, pad), 'constant', 0)
target_img = target_img_.to(device)
target_img = target_img.repeat(batch_size, 1, 1, 1)

# show image
print ('target image: ')
show_image(loaded_img)

In [ ]:
# model and optimizer
model = NCA_model(_n_channels=n_channels, _device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

# pool init
seed = Utils.make_seed(size, n_channels).to(device)
seed = nn.functional.pad(seed, (pad, pad, pad, pad), 'constant', 0)
pool = seed.clone().repeat(pool_size, 1, 1, 1)

# training loop
for it in tqdm(range(n_train_iter)):
    batch_ixs = np.random.choice(
        pool_size, batch_size, replace=False
    ).tolist()
    
    # get training batch and set first in batch as seed
    x = pool[batch_ixs]
    x[0] = seed
    
    # damage examples in batch
    if n_damage > 0:
        radius = random.uniform(size*0.1, size*0.4)
        u = random.uniform(0, 1) * size + pad
        v = random.uniform(0, 1) * size + pad
        mask = Utils.create_circle_mask(full_size, radius, [u, v])
        x[-n_damage:] *= torch.tensor(mask).to(device)
        
    # visualize batch
    if viz_train:
        show_batch(x.detach().cpu())
    
    # forward pass
    for i in range(np.random.randint(64, 96)):
        x = model(x)
        
    # visualize batch
    if viz_train:
        show_batch(x.detach().cpu())
    
    loss_batch = ((target_img - x[:, :4, ...]) ** 2).mean(dim=[1, 2, 3])
    loss = loss_batch.mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print ('train/loss: ', it, '/', loss.item())
    
    # find best in batch
    argmax_batch = loss_batch.argmax().item()
    argmax_pool = batch_ixs[argmax_batch]
    remaining_batch = [i for i in range(batch_size) if i != argmax_batch]
    remaining_pool = [i for i in batch_ixs if i != argmax_pool]
    
    pool[argmax_pool] = seed.clone()
    pool[remaining_pool] = x[remaining_batch].detach()
    
    # clear output after 100 iterations
    if it % 100 == 0:
        clear_output(wait=True)
    
    if it % eval_freq == 0:
        x_eval = seed.clone()
        eval_video = torch.empty(1, eval_freq, 3, *x_eval.shape[2:])
        for it_eval in range(eval_iter):
            x_eval = model(x_eval)
            x_eval_out = Utils.to_rgb(x_eval[:, :4].detach().cpu())
            eval_video[0, it_eval] = x_eval_out

In [ ]:
# save model
if save_model:
    model_path = pathlib.Path(model_dir)
    model_path.mkdir(parents=True, exist_ok=True)
    if name == None:
        ts = str(datetime.datetime.now()).replace(' ', '_').replace(':', '-').replace('.', '-')
        name = 'model_' + ts
    torch.save(model.state_dict(), model_dir + '\\' + name + '.pt')
    
    # save model parameters
    dict = {
        "img": img,
        "batch_size": batch_size,
        "device": device,
        "eval_freq": eval_freq,
        "eval_iter": eval_iter,
        "n_train_iter": n_train_iter,
        "n_channels": n_channels,
        "log_dir": log_dir,
        "pad": pad,
        "pool_size": pool_size,
        "size": size,
        "save_model": save_model,
        "model_dir": model_dir,
        "name": name,
        "damage": n_damage
    }
    json_object = json.dumps(dict, indent=4)
    with open(model_dir + '\\' + name + '_params.json', 'w') as outfile:
        outfile.write(json_object)